In [1]:
%%configure -f
{
    "conf": {
        "spark.jars.packages": "io.delta:delta-core_2.12:1.0.0",
        "spark.sql.extensions":"io.delta.sql.DeltaSparkSessionExtension",
        "spark.sql.catalog.spark_catalog":"org.apache.spark.sql.delta.catalog.DeltaCatalog"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,None,pyspark,starting,,,None,


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *  

spark = SparkSession.builder.appName("Lakehouse") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.databricks.delta.schema.autoMerge.enabled","true") \
    .config("spark.databricks.delta.autoOptimize.optimizeWrite","true") \
    .config("spark.databricks.delta.optimizeWrite.enabled","true") \
    .config("spark.databricks.delta.vacuum.parallelDelete.enabled","true") \
    .getOrCreate()

from delta.tables import *
from pyspark.sql.functions import *

The code failed because of a fatal error. Some things to try: a) Make sure Spark has enough available resources for Jupyter to create a Spark context. b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.   c) Restart the kernel.


### Read data from S3

In [4]:
bucket='s3://star-wars-etl-dev'
prefix='raw_data'
table='people'

The code failed because of a fatal error. Some things to try: a) Make sure Spark has enough available resources for Jupyter to create a Spark context. b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.   c) Restart the kernel.


In [5]:
df=spark.read.option("multiline","true").json(f"{bucket}/{prefix}/{table}")

The code failed because of a fatal error. Some things to try: a) Make sure Spark has enough available resources for Jupyter to create a Spark context. b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.   c) Restart the kernel.


In [6]:
df.printSchema()

The code failed because of a fatal error. Some things to try: a) Make sure Spark has enough available resources for Jupyter to create a Spark context. b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.   c) Restart the kernel.


In [79]:
df.select('count','next','previous','dat_load').show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+--------+----------+
|count|                next|previous|  dat_load|
+-----+--------------------+--------+----------+
|   82|https://swapi.dev...|    null|2023_09_20|
+-----+--------------------+--------+----------+

In [60]:
parsed_df = df.select(df['results'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
df_exploded = df.select(explode("results").alias("resultado"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
df_exploded.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- resultado: struct (nullable = true)
 |    |-- characters: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- created: string (nullable = true)
 |    |-- director: string (nullable = true)
 |    |-- edited: string (nullable = true)
 |    |-- episode_id: long (nullable = true)
 |    |-- opening_crawl: string (nullable = true)
 |    |-- planets: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- producer: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-- species: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- starships: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- title: string (nullable = true)
 |    |-- url: string (nullable = true)
 |    |-- vehicles: array (nullable = true)
 |    |    |-- element: string (containsNull = true)

In [70]:
df_exploded.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6

In [69]:
df_exploded.withColumn("resultado", col("resultado.vehicles")).show(100, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|resultado                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
+---------------------------------------------------------

In [65]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

data = [[1, "[{color: 'red', group: 'A'},{color: 'green', group: 'B'}]"],
        [2, "[{color: 'red', group: 'A'}]"]]

df = spark.createDataFrame(data, schema = ["customer_id", "model"]) \
    .withColumn("model", F.from_json("model", T.ArrayType(T.MapType(T.StringType(), T.StringType())), {"allowUnquotedFieldNames": True})) \
    .withColumn("model", F.explode("model")) \
    .withColumn("color", F.col("model")["color"]) \
    .withColumn("group", F.col("model")["group"]) \
    .drop("model")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
df.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----+-----+
|customer_id|color|group|
+-----------+-----+-----+
|          1|  red|    A|
|          1|green|    B|
|          2|  red|    A|
+-----------+-----+-----+